In [1]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D
import numpy as np
import pandas as pd

num_classes = 8
resnet_weights_path = 'imagenet'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])


In [3]:
from tensorflow.python.keras.applications.resnet import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

image_size = 224
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = data_generator.flow_from_directory(
        'output/train',
        target_size=(image_size, image_size),
        batch_size=24,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        'output/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit_generator(
        train_generator,
        steps_per_epoch=10,
        epochs = 20,
        validation_data=validation_generator,
        validation_steps=1)

Found 5105 images belonging to 8 classes.
Found 635 images belonging to 8 classes.
Epoch 1/20
 1/10 [==>...........................] - ETA: 57s - loss: 2.5974 - acc: 0.0000e+00

C:\Users\joshc\anaconda3\envs\main\lib\site-packages\PIL\Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


10/10 [==============================] - 56s 6s/step - loss: 2.0244 - acc: 0.2625 - val_loss: 1.9390 - val_acc: 0.2188
Epoch 2/20
10/10 [==============================] - 54s 5s/step - loss: 1.6587 - acc: 0.3750 - val_loss: 1.3237 - val_acc: 0.4375
Epoch 3/20
10/10 [==============================] - 55s 5s/step - loss: 1.5088 - acc: 0.4375 - val_loss: 1.4110 - val_acc: 0.4062
Epoch 4/20
10/10 [==============================] - 54s 5s/step - loss: 1.4108 - acc: 0.5042 - val_loss: 1.3439 - val_acc: 0.3750
Epoch 5/20
10/10 [==============================] - 54s 5s/step - loss: 1.2808 - acc: 0.5583 - val_loss: 0.9168 - val_acc: 0.6562
Epoch 6/20
10/10 [==============================] - 54s 5s/step - loss: 1.2093 - acc: 0.5667 - val_loss: 0.9712 - val_acc: 0.5938
Epoch 7/20
10/10 [==============================] - 55s 6s/step - loss: 1.0915 - acc: 0.6375 - val_loss: 0.8321 - val_acc: 0.7500
Epoch 8/20
10/10 [==============================] - 54s 5s/step - loss: 0.9632 - acc: 0.6667 - val_lo

In [ ]:
test_generator = data_generator.flow_from_directory(
        'output/test',
        target_size=(image_size, image_size),
        batch_size=24,
        class_mode='categorical')

In [ ]:
pred = my_new_model.predict(test_generator)


In [ ]:
pred


In [ ]:
np.save('class_indices1000.npy',train_generator.class_indices)

In [ ]:
import numpy as np
import pandas as pd
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
filenames=test_generator.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Predictions":predictions})
results.head(20)

In [ ]:
my_new_model.save("model1000")

In [ ]:
from tensorflow.python.keras.models import load_model

model1 = load_model('multpleFish')

In [ ]:
model1.predict(test_generator)

In [ ]:
import glob
list = []
for filepath in glob.iglob(r'C:\Users\joshc\OneDrive\Desktop\test\*.png'):
    list.append(filepath)
print(list)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
for filepath in list:
    image =cv2.imread(filepath)

    ###change color of image to be displayed by matplotlib
    im2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    ###expand from 3 to 4 dimesion
    img1 = np.expand_dims(image, axis=0)

    ##convert to an numpy arr
    test = np.array(im2)

    ##predict the image
    result = model1.predict(img1)

    ##corespoding label integer
    y_classes = np.argmax(result, axis=1)

    ###predicting on the image
    single_pred = np.squeeze(model1.predict(img1))


    ##get labels
    labels = np.load('class_indicies' + '.npy', allow_pickle=True).item()
    class_indices = labels

    ###
    labels = dict((v,k) for k,v in labels.items())
    prediction = [labels[k] for k in y_classes]
    results=pd.DataFrame({"Predictions":prediction})


    ##get percents
    results_dict = dict(zip(class_indices.keys(), single_pred))


    ##Saves prediction as a string
    prediction = prediction.pop(0)

    print(results_dict)
    plt.imshow(test)
    plt.title(prediction)
    plt.show()

    cv2.imshow(prediction, image)


###Ends program
while True:
    key = cv2.waitKey(1)
    
    if key == ord('q'):
        break
cv2.destroyAllWindows()


In [ ]:
f, ax = plt.subplots(5, 5, figsize = (15, 15))
i =0
for filename in list:
    image =cv2.imread(filename)
    
    ###change color of image to be displayed by matplotlib
    im2 = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    ###expand from 3 to 4 dimesion
    img1 = np.expand_dims(image, axis=0)

    ##convert to an numpy arr
    test = np.array(im2)

    ##predict the image
    result = model1.predict(img1)

    ##corespoding label integer
    y_classes = np.argmax(result, axis=1)

    ###predicting on the image
    single_pred = np.squeeze(model1.predict(img1))


    ##get labels
    labels = np.load('class_indicies' + '.npy', allow_pickle=True).item()
    class_indices = labels

    ###
    labels = dict((v,k) for k,v in labels.items())
    prediction = [labels[k] for k in y_classes]
    results=pd.DataFrame({"Predictions":prediction})


    ##get percents
    results_dict = dict(zip(class_indices.keys(), single_pred))


    ##Saves prediction as a string
    prediction = prediction.pop(0)

    ax[i//5, i%5].imshow(im2)
    ax[i//5, i%5].axis('off')
    ax[i//5, i%5].set_title("Predicted:{}".format(prediction))    
    i+=1
plt.show()